Collects results MSE and R2 and create report

Fedot 

In [160]:
import os

import numpy as np
import pandas as pd
import pickle 

from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Plots
import matplotlib.pyplot as plt

# Prerocessing for FEDOT
from fedot.core.data.data import InputData
from fedot.core.repository.dataset_types import DataTypesEnum
from fedot.api.main import Fedot
from fedot.core.repository.tasks import Task, TaskTypesEnum, TsForecastingParams

# FEDOT 
from fedot.core.pipelines.pipeline import Pipeline
from fedot.core.pipelines.node import PrimaryNode, SecondaryNode

import logging
logging.raiseExceptions = False

df = None
with open("covid_flow.pkl", "rb") as f:
    df = pickle.load(f)
df["case"] = df.index
df["t_point"] = df["t_point"].apply(lambda x: x[2:])
df["t_point"] = df["t_point"].apply(lambda x: int(x))
df = df.set_index(["case", "t_point"])
df = df.sort_values(["case", "t_point"])

def prepare_data_with_dynamic_window(df_input, window=1, test_size=0.2):
    X, y = [], []
    df = df_input.copy()
    df_grouped = df.groupby(["case"]).size()
    df_idx = df_grouped[df_grouped>=window+1].index
    for i in df_idx:
        for j in range(len(df.loc[(i)])-window):
            wind = df.loc[(i, j): (i, j+window-1)]
            X.append(wind.iloc[:, :47])
            y.append(df.loc[(i, j+window)].iloc[29:42])
    X, y = np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return (X_train, y_train), (X_test, y_test)

scores = []
for WINDOW in range(1, 5):
    (X_train, y_train), (X_test, y_test) = prepare_data_with_dynamic_window(df, window=WINDOW)
    print("Test train shapes:", X_train.shape,
    X_test.shape)
    X_train = X_train.reshape(-1, WINDOW*47)
    X_test = X_test.reshape(-1, WINDOW*47)

    models = {}
    dinam_fact_columns = df.columns[29:42]
    for param_idx, param_name in enumerate(dinam_fact_columns):
        try:
            model = Fedot("regression")
            model.load(f"fedot_w{WINDOW}\\fedot_regress_one_window_param_{param_idx}\\fedot_regress_one_window_param_{param_idx}.json")
            y_pred = model.predict(X_test)
            mask = ~np.isnan(y_test[:, param_idx])
            y_check = y_test[mask, param_idx] 
            y_pred = y_pred[mask] 
            scaler = MinMaxScaler()

            scaler.fit(y_check.reshape(-1, 1))
            y_pred = scaler.transform(y_pred.reshape(-1, 1)).reshape(-1)
            y_check = scaler.transform(y_check.reshape(-1, 1)).reshape(-1)
            r2_res = r2_score(y_check, y_pred, multioutput="raw_values")
            mse = mean_squared_error(y_check, y_pred)
            scores.append([param_name, WINDOW, mse, r2_res[0]])
        except Exception as e:
            print(f"FAILED {param_name} {e}")

df = pd.DataFrame(scores, columns = ["Параметр", "Окно", "MSE", "R2"])
df = df.set_index(["Параметр", "Окно"])
df = df.sort_values(["Параметр", "Окно"])
df

ws = [{} for i in range(1, 5)]
for i in range(len(scores)):
    ws[scores[i][1]-1][scores[i][0]] = [scores[i][2], scores[i][3]]
for j in range(4):
    df = pd.DataFrame(ws[j], index=["MSE", "R2"])
    df.to_csv(f"reports_w{j+1}\\fedot.csv")

Test train shapes: (3356, 1, 47) (840, 1, 47)
Test train shapes: (1805, 2, 47) (452, 2, 47)
Test train shapes: (712, 3, 47) (178, 3, 47)
Test train shapes: (288, 4, 47) (72, 4, 47)


Neural Networks

In [161]:
import pandas as pd
import os
def report(WINDOW, PARAM, fedot=False):
    reps = os.listdir(f"reports_w{WINDOW}")
    k = []
    df = pd.DataFrame()
    for i in reps:
        if (not fedot and i == "fedot.csv"):
            continue
        mname = i.split(".")[0]
        dfix = pd.read_csv(f"reports_w{WINDOW}\\{i}")
        dfix = dfix[dfix["Unnamed: 0"]==PARAM]
        dfix["model"] = [mname]
        dfix["metric"] = dfix["Unnamed: 0"]
        dfix["window"] = WINDOW
        dfix = dfix.drop("Unnamed: 0", axis=1)
        dfix = dfix.set_index(["model", "metric", "window"])
        dfix = dfix.sort_index()
        df = pd.concat([df, dfix])
        k = dfix
    print("Window ", WINDOW, ".", "Statistic: ", PARAM)
    return df

In [162]:
def summary_on_window(WINDOW, fedot=False):
    df = report(WINDOW, "R2", fedot)
    dfms = report(WINDOW, "MSE", fedot)
    print("\n")
    g = df.idxmax().dropna()
    r = df.max().dropna()
    sdf = []
    for i in range(len(g)):
        sdf.append([g.index[i], g[i][0], r[i], dfms.loc[(g[i][0], "MSE", WINDOW)][g.index[i]]])
    r = pd.DataFrame(sdf, columns=["Параметр", "Модель", "R2", "MSE"])
    r = r.set_index(["Параметр"])
    return r

Window 1

In [163]:
report(1, "R2")

Window  1 . Statistic:  R2


,,,Температура_dinam_fact,Лимфоциты#_dinam_fact,АСТ_dinam_fact,ЧСС_dinam_fact,ЧДД_dinam_fact,Билирубин общий_dinam_fact,MPV- Средн.объем тромбоцитов_dinam_fact,PCT- Тромбокрит_dinam_fact,Лимфоциты%_dinam_fact,снижение_сознания_dinam_fact,Cтепень тяжести по КТ_dinam_fact,Лактатдегидрогеназа_dinam_fact,PDW- Индекс расп.по объему тр._dinam_fact
model,metric,window,,,,,,,,,,,,,
Faltten1Layer,R2,1,-0.153128,0.824537,0.449263,0.004991,0.064152,-0.029547,0.530503,0.314895,0.581243,0.840004,-0.031280,0.191407,0.551577
Faltten2Layer,R2,1,-0.047552,0.021497,-3.548916,-0.002270,0.126297,0.254766,0.456608,0.236184,0.356584,0.898571,0.005621,0.121574,0.597463
Flatten2Layers_Mixed,R2,1,-0.129344,-7.735878,-0.176490,-0.054341,-0.047593,0.034465,0.567352,0.268179,0.273611,0.895129,0.018438,-0.114441,0.551620
Flatten3Layers,R2,1,-0.018610,0.646399,-0.028205,0.001001,0.260884,0.397709,0.639676,0.273760,0.412794,0.898233,0.031437,0.091414,0.631905
Flatten3Layers_batch2,R2,1,0.002703,0.654899,-0.217651,-0.024305,0.299234,0.362046,0.650920,0.297529,0.335726,0.904208,-0.001999,0.081084,0.635137
Flatten3Layers_batch4,R2,1,-0.018152,0.483068,0.313549,-0.012326,0.144792,0.361937,0.630448,0.286082,0.405361,0.889357,0.028335,0.098203,0.596647
Flatten3Layers_batch8,R2,1,-0.120533,0.404619,0.034927,0.010583,0.342372,0.384406,0.637670,0.293007,0.418218,0.894800,0.009306,0.099181,0.596753
Flatten3Layers_Mixed,R2,1,-0.041336,-10.366083,-0.933684,-0.026723,0.085911,-0.487080,0.553137,0.230948,0.262584,0.908189,-0.001277,-0.120662,0.494984
Flatten4Layers,R2,1,-0.143513,0.571339,-0.252122,-0.030203,-0.000187,0.319314,0.630253,0.237107,0.369893,0.902174,0.030294,-0.002377,0.642589


In [164]:
summary_on_window(1)

Window  1 . Statistic:  R2
Window  1 . Statistic:  MSE




,Модель,R2,MSE
Параметр,,,
Температура_dinam_fact,Flatten5Layers_batch4,0.010550,0.002376
Лимфоциты#_dinam_fact,Faltten1Layer,0.824537,0.001349
АСТ_dinam_fact,Faltten1Layer,0.449263,0.000157
ЧСС_dinam_fact,Flatten3Layers_batch8,0.010583,0.008367
ЧДД_dinam_fact,Flatten5Layers_batch4,0.344241,0.000147
Билирубин общий_dinam_fact,Flatten3Layers,0.397709,0.001319
MPV- Средн.объем тромбоцитов_dinam_fact,Flatten5Layers_batch4,0.652261,0.005411
PCT- Тромбокрит_dinam_fact,Faltten1Layer,0.314895,0.008465
Лимфоциты%_dinam_fact,Faltten1Layer,0.581243,0.009657


Window 2

In [165]:
summary_on_window(2)

Window  2 . Statistic:  R2
Window  2 . Statistic:  MSE




,Модель,R2,MSE
Параметр,,,
Температура_dinam_fact,GRU13_Batch8,0.024583,0.001804
Лимфоциты#_dinam_fact,StaticDense_DynamicLSTM_twoLayers,0.661245,0.001363
АСТ_dinam_fact,GRU64_GRU32_Dense,0.604499,0.000167
ЧСС_dinam_fact,RNNMixed,0.050058,0.007378
ЧДД_dinam_fact,LSTMtoLSTM_Mixed,0.413600,0.000145
Билирубин общий_dinam_fact,LSTM64Mixed,0.361337,0.000881
MPV- Средн.объем тромбоцитов_dinam_fact,GRU32xGRU32_Batch4,0.651195,0.004539
PCT- Тромбокрит_dinam_fact,GRU32xGRU32_Batch4,0.449788,0.004105
Лимфоциты%_dinam_fact,StaticDense_DynamicLSTM_twoLayers,0.596125,0.007296


In [166]:
report(2, "R2")

Window  2 . Statistic:  R2


,,,Температура_dinam_fact,Лимфоциты#_dinam_fact,АСТ_dinam_fact,ЧСС_dinam_fact,ЧДД_dinam_fact,Билирубин общий_dinam_fact,MPV- Средн.объем тромбоцитов_dinam_fact,PCT- Тромбокрит_dinam_fact,Лимфоциты%_dinam_fact,снижение_сознания_dinam_fact,Cтепень тяжести по КТ_dinam_fact,Лактатдегидрогеназа_dinam_fact,PDW- Индекс расп.по объему тр._dinam_fact
model,metric,window,,,,,,,,,,,,,
Faltten1Layer,R2,2,-0.334506,-1.487261,-0.120240,-0.015271,0.165023,0.094596,0.536068,0.349104,0.535053,0.993618,0.004136,-0.011637,0.648669
Flatten3Layers,R2,2,-0.396342,-7.615566,-0.163404,-0.086933,-0.593602,-0.183020,0.464412,0.139971,0.371105,0.971947,0.009107,-0.031460,0.505675
Flatten3Layers_Mixed,R2,2,-0.289751,-18.547484,-0.081369,0.012508,-0.270599,0.060651,0.424120,0.211812,0.384191,0.789162,-0.095096,-0.108671,0.535222
GRU13,R2,2,-0.123922,-4.107154,-0.055887,-0.039932,0.041240,0.105580,0.572444,0.299942,0.539216,0.992283,-0.007351,0.044901,0.684199
GRU13_Batch2,R2,2,-0.113127,0.193976,-1.447290,-0.018912,-0.308173,-0.231517,0.639926,0.388333,0.440939,0.756718,-0.010840,-1.408281,0.606479
GRU13_Batch4,R2,2,-0.143159,0.093168,0.428257,-0.003283,0.363181,-0.216959,0.641636,0.416631,0.430338,0.756321,-0.008768,0.367211,0.574922
GRU13_Batch8,R2,2,0.024583,0.257805,-0.084929,0.012798,0.208425,0.087445,0.588853,0.308011,0.552696,0.993738,0.013579,0.068529,0.699062
GRU32xGRU32_Batch4,R2,2,-0.045923,-2.749460,0.595024,-0.027583,0.390174,-0.059511,0.651195,0.449788,0.417105,0.759017,-0.018922,0.408160,0.605970
GRU60Mixed,R2,2,-0.164659,-9.925552,-0.720276,0.030390,-0.121120,0.282828,0.504949,0.309147,0.450109,0.803836,-0.048593,0.103211,0.643061


Window 3

In [167]:
summary_on_window(3)

Window  3 . Statistic:  R2
Window  3 . Statistic:  MSE




,Модель,R2,MSE
Параметр,,,
Температура_dinam_fact,SimpleRNN13_Tanh,-0.025246,0.001267
Лимфоциты#_dinam_fact,LSTM64xLSTM32_Batch4,0.991304,0.000082
АСТ_dinam_fact,LSTM13_Batch2,0.701792,0.000217
ЧСС_dinam_fact,LSTM64xLSTM32_Batch4,-0.036390,0.011386
ЧДД_dinam_fact,LSTM13_Batch1,0.634257,0.000103
Билирубин общий_dinam_fact,LSTM64_LSTM32_Dense,0.283741,0.001486
MPV- Средн.объем тромбоцитов_dinam_fact,LSTM13_Batch8,0.721695,0.003346
PCT- Тромбокрит_dinam_fact,LSTM64xLSTM32_Batch4,0.195863,0.005125
Лимфоциты%_dinam_fact,LSTM64xLSTM32_Batch4,0.639118,0.006520


In [168]:
report(3, "R2")

Window  3 . Statistic:  R2


,,,Температура_dinam_fact,Лимфоциты#_dinam_fact,АСТ_dinam_fact,ЧСС_dinam_fact,ЧДД_dinam_fact,Билирубин общий_dinam_fact,MPV- Средн.объем тромбоцитов_dinam_fact,PCT- Тромбокрит_dinam_fact,Лимфоциты%_dinam_fact,снижение_сознания_dinam_fact,Cтепень тяжести по КТ_dinam_fact,Лактатдегидрогеназа_dinam_fact,PDW- Индекс расп.по объему тр._dinam_fact
model,metric,window,,,,,,,,,,,,,
Faltten1Layer,R2,3,-1.247323,0.698661,-0.978406,-0.246791,-0.519388,-0.232048,0.662094,0.005256,0.569456,0.982839,-0.097756,-0.704417,0.505689
Flatten3Layers,R2,3,-0.667435,0.176988,0.275951,-0.225525,0.037945,-0.008984,0.315952,-0.019594,0.499369,0.987608,-0.047775,-0.048160,0.066522
GRU13,R2,3,-0.395437,0.624420,0.429790,-0.375197,-0.078367,0.270063,0.559090,0.022819,0.601012,0.992006,-0.060690,0.175208,0.618453
GRU13_Batch8,R2,3,-0.225563,0.779008,0.330161,-0.165413,0.047547,0.054310,0.702216,0.141907,0.613061,0.994738,-0.188758,-0.584016,0.599343
LSTM13,R2,3,-0.161715,0.476840,-0.084256,-0.374281,-0.019889,0.232829,0.284742,0.035953,0.581803,0.990644,-0.045297,0.471150,0.302665
LSTM13_Batch1,R2,3,-0.280880,0.961950,0.590428,-0.272761,0.634257,-0.864045,0.600128,0.175612,0.620248,0.988438,-0.135998,0.336498,0.609307
LSTM13_Batch2,R2,3,-0.231551,0.979963,0.701792,-0.347858,0.354564,-0.297045,0.719808,0.191574,0.638230,0.992868,-0.336815,0.311495,0.639641
LSTM13_Batch4,R2,3,-0.065043,0.937955,-0.107709,-0.190250,0.247733,0.096168,0.664721,0.167466,0.635875,0.990460,-0.308831,0.019972,0.656922
LSTM13_Batch8,R2,3,-0.025273,0.860335,0.394310,-0.188571,0.189043,0.111199,0.721695,0.150201,0.593590,0.986785,-0.074954,0.257379,0.652576


Window 4

In [169]:
summary_on_window(4)

Window  4 . Statistic:  R2
Window  4 . Statistic:  MSE




,Модель,R2,MSE
Параметр,,,
Температура_dinam_fact,GRU13_Batch1,0.017721,0.001471
Лимфоциты#_dinam_fact,GRU32_Batch4,0.602346,0.009426
АСТ_dinam_fact,LSTM64xLSTM13_Batch4,0.134370,0.000607
ЧСС_dinam_fact,LSTM13_Batch4,0.071622,0.012290
ЧДД_dinam_fact,LSTM13_Batch8,0.404697,0.000176
Билирубин общий_dinam_fact,GRU13_Batch4,0.322373,0.002310
MPV- Средн.объем тромбоцитов_dinam_fact,GRU32_Batch4,0.670607,0.003386
PCT- Тромбокрит_dinam_fact,GRU13_Batch2,0.411801,0.002133
Лимфоциты%_dinam_fact,GRU32xGRU32_Batch4,0.754267,0.005626


In [170]:
report(4, "R2")

Window  4 . Statistic:  R2


,,,Температура_dinam_fact,Лимфоциты#_dinam_fact,АСТ_dinam_fact,ЧСС_dinam_fact,ЧДД_dinam_fact,Билирубин общий_dinam_fact,MPV- Средн.объем тромбоцитов_dinam_fact,PCT- Тромбокрит_dinam_fact,Лимфоциты%_dinam_fact,снижение_сознания_dinam_fact,Cтепень тяжести по КТ_dinam_fact,Лактатдегидрогеназа_dinam_fact,PDW- Индекс расп.по объему тр._dinam_fact
model,metric,window,,,,,,,,,,,,,
Faltten1Layer,R2,4,-0.635448,0.303339,-0.904954,-0.146198,-4.743965,-0.296898,-0.136606,0.045420,0.030100,0.790680,-0.101754,-2.133585,0.151376
Flatten3Layers,R2,4,-1.408934,0.015252,-0.261115,-0.068230,-21.850275,0.033659,-0.149225,-1.226264,0.253065,0.509053,-0.110927,-2.950291,-0.674690
Flatten3Layers_Mixed,R2,4,-3.730922,-128.922573,-6.195192,-0.831238,-1.881025,-2.627866,0.011026,-0.486396,0.293515,0.800226,-0.762103,-2.573944,0.515039
Flatten3Layers_Mixed_flatten,R2,4,-2.058989,-83.285055,-3.009570,-0.758005,-7.544391,-4.962606,0.045501,-0.301305,0.087104,0.802257,-0.680280,-1.155848,0.481299
GRU13,R2,4,-2.301274,0.340789,-1.691932,-0.257493,-7.632477,-0.378725,-0.204954,-0.426899,0.214278,0.754478,-0.465426,-2.039608,-0.349623
GRU13_Batch1,R2,4,0.017721,0.549973,-0.176101,0.005445,0.298628,0.008594,0.665743,0.375204,0.634813,0.806279,0.138239,-0.121908,0.511942
GRU13_Batch2,R2,4,-0.074959,0.329207,-1.671331,-0.014999,-0.246091,0.121106,0.558071,0.411801,0.721429,0.813836,0.024219,0.290591,0.510645
GRU13_Batch4,R2,4,-0.918578,0.424006,-0.322748,-0.068319,0.300805,0.322373,0.470003,-0.307578,0.620341,0.816963,0.022044,-0.855032,0.505989
GRU13_Batch8,R2,4,-0.614212,0.269406,-0.273613,-0.340211,-1.272181,0.160740,0.329257,0.090412,0.637480,0.835655,-0.013179,-1.282880,0.491184


Overall summary

In [171]:
summary_on_window(2)

Window  2 . Statistic:  R2
Window  2 . Statistic:  MSE




,Модель,R2,MSE
Параметр,,,
Температура_dinam_fact,GRU13_Batch8,0.024583,0.001804
Лимфоциты#_dinam_fact,StaticDense_DynamicLSTM_twoLayers,0.661245,0.001363
АСТ_dinam_fact,GRU64_GRU32_Dense,0.604499,0.000167
ЧСС_dinam_fact,RNNMixed,0.050058,0.007378
ЧДД_dinam_fact,LSTMtoLSTM_Mixed,0.413600,0.000145
Билирубин общий_dinam_fact,LSTM64Mixed,0.361337,0.000881
MPV- Средн.объем тромбоцитов_dinam_fact,GRU32xGRU32_Batch4,0.651195,0.004539
PCT- Тромбокрит_dinam_fact,GRU32xGRU32_Batch4,0.449788,0.004105
Лимфоциты%_dinam_fact,StaticDense_DynamicLSTM_twoLayers,0.596125,0.007296


Result NN

In [172]:
params = {}
for win in range(1, 5):
    dfx = summary_on_window(win)
    for p in dfx.index:
        if (p not in params):
            params[p] = [win, dfx.loc[p]["Модель"], dfx.loc[p]["R2"], dfx.loc[p]["MSE"]]
        else:
            if (params[p][2] < dfx.loc[p]["R2"]):
                params[p] = [win, dfx.loc[p]["Модель"], dfx.loc[p]["R2"], dfx.loc[p]["MSE"]]
                
result_nn = pd.DataFrame(params.values(), columns=["Окно", *dfx.columns], index=params.keys())
result_nn = result_nn.sort_values("R2", ascending=False)
result_nn

Window  1 . Statistic:  R2
Window  1 . Statistic:  MSE


Window  2 . Statistic:  R2
Window  2 . Statistic:  MSE


Window  3 . Statistic:  R2
Window  3 . Statistic:  MSE


Window  4 . Statistic:  R2
Window  4 . Statistic:  MSE




,Окно,Модель,R2,MSE
снижение_сознания_dinam_fact,2,LSTM13_Batch4,0.997207,0.000108
Лимфоциты#_dinam_fact,3,LSTM64xLSTM32_Batch4,0.991304,0.000082
Лимфоциты%_dinam_fact,4,GRU32xGRU32_Batch4,0.754267,0.005626
PDW- Индекс расп.по объему тр._dinam_fact,4,StaticDense_DynamicGRU_TwoLayersResult,0.727560,0.003408
MPV- Средн.объем тромбоцитов_dinam_fact,3,LSTM13_Batch8,0.721695,0.003346
АСТ_dinam_fact,3,LSTM13_Batch2,0.701792,0.000217
ЧДД_dinam_fact,3,LSTM13_Batch1,0.634257,0.000103
Лактатдегидрогеназа_dinam_fact,3,LSTM64xLSTM32_Batch4,0.499811,0.000392
PCT- Тромбокрит_dinam_fact,2,GRU32xGRU32_Batch4,0.449788,0.004105
Билирубин общий_dinam_fact,1,Flatten3Layers,0.397709,0.001319


Fedot

In [173]:
import os

import numpy as np
import pandas as pd
import pickle 

from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Plots
import matplotlib.pyplot as plt

# Prerocessing for FEDOT
from fedot.core.data.data import InputData
from fedot.core.repository.dataset_types import DataTypesEnum
from fedot.api.main import Fedot
from fedot.core.repository.tasks import Task, TaskTypesEnum, TsForecastingParams

# FEDOT 
from fedot.core.pipelines.pipeline import Pipeline
from fedot.core.pipelines.node import PrimaryNode, SecondaryNode

import logging
logging.raiseExceptions = False

df = None
with open("covid_flow.pkl", "rb") as f:
    df = pickle.load(f)
df["case"] = df.index
df["t_point"] = df["t_point"].apply(lambda x: x[2:])
df["t_point"] = df["t_point"].apply(lambda x: int(x))
df = df.set_index(["case", "t_point"])
df = df.sort_values(["case", "t_point"])

def prepare_data_with_dynamic_window(df_input, window=1, test_size=0.2):
    X, y = [], []
    df = df_input.copy()
    df_grouped = df.groupby(["case"]).size()
    df_idx = df_grouped[df_grouped>=window+1].index
    for i in df_idx:
        for j in range(len(df.loc[(i)])-window):
            wind = df.loc[(i, j): (i, j+window-1)]
            X.append(wind.iloc[:, :47])
            y.append(df.loc[(i, j+window)].iloc[29:42])
    X, y = np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return (X_train, y_train), (X_test, y_test)

scores = []
for WINDOW in range(1, 5):
    (X_train, y_train), (X_test, y_test) = prepare_data_with_dynamic_window(df, window=WINDOW)
    print("Test train shapes:", X_train.shape,
    X_test.shape)
    X_train = X_train.reshape(-1, WINDOW*47)
    X_test = X_test.reshape(-1, WINDOW*47)

    models = {}
    dinam_fact_columns = df.columns[29:42]
    for param_idx, param_name in enumerate(dinam_fact_columns):
        try:
            model = Fedot("regression")
            model.load(f"fedot_w{WINDOW}\\fedot_regress_one_window_param_{param_idx}\\fedot_regress_one_window_param_{param_idx}.json")
            y_pred = model.predict(X_test)
            mask = ~np.isnan(y_test[:, param_idx])
            y_check = y_test[mask, param_idx] 
            y_pred = y_pred[mask] 
            scaler = MinMaxScaler()

            scaler.fit(y_check.reshape(-1, 1))
            y_pred = scaler.transform(y_pred.reshape(-1, 1)).reshape(-1)
            y_check = scaler.transform(y_check.reshape(-1, 1)).reshape(-1)
            r2_res = r2_score(y_check, y_pred, multioutput="raw_values")
            mse = mean_squared_error(y_check, y_pred)
            scores.append([param_name, WINDOW, mse, r2_res[0]])
        except Exception as e:
            print(f"FAILED {param_name} {e}")

df = pd.DataFrame(scores, columns = ["Параметр", "Окно", "MSE", "R2"])
df = df.set_index(["Параметр", "Окно"])
df = df.sort_values(["Параметр", "Окно"])

df_complex = pd.DataFrame()

ws = [{} for i in range(1, 5)]
for i in range(len(scores)):
    ws[scores[i][1]-1][scores[i][0]] = [scores[i][2], scores[i][3]]
for j in range(4):
    df = pd.DataFrame(ws[j], index=["MSE", "R2"])
    df.to_csv(f"reports_w{j+1}\\fedot.csv")
    df_complex = pd.concat([df_complex, df])
df_complex.index= list(range(8))
df_complex["window"] = 0
df_complex["metric"] = 0
for i in range(8):
    df_complex.loc[i, "window"] = i//2+1
    if(i%2==0):
        df_complex.loc[i, "metric"] = "MSE"
    else:
        df_complex.loc[i, "metric"] = "R2"
df_complex = df_complex.set_index(["window", "metric"])
df_complex = df_complex.sort_values(["window", "metric"])
df_complex

Test train shapes: (3356, 1, 47) (840, 1, 47)
Test train shapes: (1805, 2, 47) (452, 2, 47)
Test train shapes: (712, 3, 47) (178, 3, 47)
Test train shapes: (288, 4, 47) (72, 4, 47)


Температура_dinam_fact  Лимфоциты#_dinam_fact  АСТ_dinam_fact  \
window metric                                                                  
1      MSE                   0.004965               0.000067        0.001513   
       R2                    0.259328               0.973649        0.181626   
2      MSE                   0.006520               0.000099        0.008252   
       R2                    0.150635               0.965210       -0.003914   
3      MSE                   0.011782               0.031235        0.015494   
       R2                    0.545331               0.370250        0.080724   
4      MSE                   0.015320               0.029704        0.186066   
       R2                    0.608237               0.116703       -2.987740   

               ЧСС_dinam_fact  ЧДД_dinam_fact  Билирубин общий_dinam_fact  \
window metric                                                               
1      MSE           0.014853        0.561032                    0.004989   
       R2            0.033455      -83.015804                    0.387925   
2      MSE           0.008581        0.002352                    0.001483   
       R2            0.581279        0.497280                    0.799135   
3      MSE           0.025760        2.538668                    0.010551   
       R2           -0.009031      -94.507720                    0.416871   
4      MSE           0.003261        0.011515                    0.004866   
       R2            0.881666        0.681255                    0.867605   

               MPV- Средн.объем тромбоцитов_dinam_fact  \
window metric                                            
1      MSE                                    0.004757   
       R2                                     0.806688   
2      MSE                                    0.005501   
       R2                                     0.815172   
3      MSE                                    0.010495   
       R2                                     0.784906   
4      MSE                                    0.004195   
       R2                                     0.912909   

               PCT- Тромбокрит_dinam_fact  Лимфоциты%_dinam_fact  \
window metric                                                      
1      MSE                       0.007961               0.006719   
       R2                        0.535645               0.643946   
2      MSE                       0.013469               0.006229   
       R2                        0.461043               0.696388   
3      MSE                       0.011421               0.019359   
       R2                        0.609137               0.582793   
4      MSE                       0.014774               0.017625   
       R2                        0.735104               0.508086   

               снижение_сознания_dinam_fact  Cтепень тяжести по КТ_dinam_fact  \
window metric                                                                   
1      MSE                         0.004782                          0.059169   
       R2                          0.899321                          0.186165   
2      MSE                         0.005766                          0.071909   
       R2                          0.906946                          0.158531   
3      MSE                       330.982248                          0.028965   
       R2                      -4288.096622                          0.342180   
4      MSE                         0.014403                          0.040930   
       R2                          0.888723                          0.231052   

               Лактатдегидрогеназа_dinam_fact  \
window metric                                   
1      MSE                           0.008475   
       R2                            0.108102   
2      MSE                           0.004804   
       R2                            0.344950   
3      MSE                           0.006396   
       R2                  

Fedot max result

In [216]:
data = []
for i in df_complex.columns:
    maxr2 = -1000
    maxmse = 0
    mod = None
    wind = 1
    for k in range(1, 5):
        r2 = df_complex[i].loc[k, "R2"]
        if (r2 > maxr2):
            maxr2 = r2
            wind = k
            maxmse = df_complex[i].loc[k, "MSE"]
    data.append([i, wind, maxr2, maxmse])
data = pd.DataFrame(data, columns=["Параметр", "Окно", "R2", "MSE"])
data =data.set_index(["Параметр"])
df_fedot = data.sort_values(["R2"], ascending=False)
df_fedot

,Окно,R2,MSE
Параметр,,,
Лимфоциты#_dinam_fact,1,0.973649,0.000067
PDW- Индекс расп.по объему тр._dinam_fact,3,0.918739,0.004195
MPV- Средн.объем тромбоцитов_dinam_fact,4,0.912909,0.004195
снижение_сознания_dinam_fact,2,0.906946,0.005766
Лактатдегидрогеназа_dinam_fact,4,0.906615,0.007804
ЧСС_dinam_fact,4,0.881666,0.003261
Билирубин общий_dinam_fact,4,0.867605,0.004866
PCT- Тромбокрит_dinam_fact,4,0.735104,0.014774
Лимфоциты%_dinam_fact,2,0.696388,0.006229


Comapre NN and Fedot

In [217]:
df_fedot["Модель_нейросеть"] = None
df_fedot["R2_нейросеть"] = None
df_fedot["MSE_нейросеть"] = None
for i in df_fedot.index:
    df_fedot.loc[i, "R2_нейросеть"] = result_nn.loc[i, "R2"]
    df_fedot.loc[i, "Модель_нейросеть"] = result_nn.loc[i, "Модель"]
    df_fedot.loc[i, "Окно_нейросеть"] = result_nn.loc[i, "Окно"]
    df_fedot.loc[i, "MSE_нейросеть"] = result_nn.loc[i, "MSE"]
df_fedot

,Окно,R2,MSE,Модель_нейросеть,R2_нейросеть,MSE_нейросеть,Окно_нейросеть
Параметр,,,,,,,
Лимфоциты#_dinam_fact,1,0.973649,0.000067,LSTM64xLSTM32_Batch4,0.991304,0.000082,3.0
PDW- Индекс расп.по объему тр._dinam_fact,3,0.918739,0.004195,StaticDense_DynamicGRU_TwoLayersResult,0.72756,0.003408,4.0
MPV- Средн.объем тромбоцитов_dinam_fact,4,0.912909,0.004195,LSTM13_Batch8,0.721695,0.003346,3.0
снижение_сознания_dinam_fact,2,0.906946,0.005766,LSTM13_Batch4,0.997207,0.000108,2.0
Лактатдегидрогеназа_dinam_fact,4,0.906615,0.007804,LSTM64xLSTM32_Batch4,0.499811,0.000392,3.0
ЧСС_dinam_fact,4,0.881666,0.003261,LSTM13_Batch4,0.071622,0.01229,4.0
Билирубин общий_dinam_fact,4,0.867605,0.004866,Flatten3Layers,0.397709,0.001319,1.0
PCT- Тромбокрит_dinam_fact,4,0.735104,0.014774,GRU32xGRU32_Batch4,0.449788,0.004105,2.0
Лимфоциты%_dinam_fact,2,0.696388,0.006229,GRU32xGRU32_Batch4,0.754267,0.005626,4.0


In [264]:
%matplotlib inline
for idx in df_fedot.index:
    paramidx = dinam_fact_columns.to_list().index(idx)
    wind = df_fedot.loc[idx, "Окно"]
    pipe_path = "fedot_w{}\\fedot_regress_one_window_param_{}\\fedot_regress_one_window_param_{}.json".format(wind, param_idx, param_idx)
    obtained_pipline = Pipeline()
    obtained_pipline.load(pipe_path)
    display(idx, "R2",  df_fedot.loc[idx, "R2"])
    obtained_pipline.show(f"{idx}.jpg")

'Лимфоциты#_dinam_fact'

'R2'

0.9736491953318713

'PDW- Индекс расп.по объему тр._dinam_fact'

'R2'

...

'R2'

0.34217953505729937

'АСТ_dinam_fact'

'R2'

0.18162645950158662

In [252]:
obtained_pipline.show("a.jpg")


Final report

In [218]:
params = {}
for win in range(1, 5):
    dfx = summary_on_window(win, fedot=True)
    for p in dfx.index:
        if (p not in params):
            params[p] = [win, dfx.loc[p]["Модель"], dfx.loc[p]["R2"], dfx.loc[p]["MSE"]]
        else:
            if (params[p][2] < dfx.loc[p]["R2"]):
                params[p] = [win, dfx.loc[p]["Модель"], dfx.loc[p]["R2"], dfx.loc[p]["MSE"]]
                
result = pd.DataFrame(params.values(), columns=["Окно", *dfx.columns], index=params.keys())
result = result.sort_values("R2", ascending=False)
result.to_csv("Final_fesult_work.csv")
result

Window  1 . Statistic:  R2
Window  1 . Statistic:  MSE


Window  2 . Statistic:  R2
Window  2 . Statistic:  MSE


Window  3 . Statistic:  R2
Window  3 . Statistic:  MSE


Window  4 . Statistic:  R2
Window  4 . Statistic:  MSE




,Окно,Модель,R2,MSE
снижение_сознания_dinam_fact,2,LSTM13_Batch4,0.997207,0.000108
Лимфоциты#_dinam_fact,3,LSTM64xLSTM32_Batch4,0.991304,0.000082
PDW- Индекс расп.по объему тр._dinam_fact,3,fedot,0.918739,0.004195
MPV- Средн.объем тромбоцитов_dinam_fact,4,fedot,0.912909,0.004195
Лактатдегидрогеназа_dinam_fact,4,fedot,0.906615,0.007804
ЧСС_dinam_fact,4,fedot,0.881666,0.003261
Билирубин общий_dinam_fact,4,fedot,0.867605,0.004866
Лимфоциты%_dinam_fact,4,GRU32xGRU32_Batch4,0.754267,0.005626
PCT- Тромбокрит_dinam_fact,4,fedot,0.735104,0.014774
АСТ_dinam_fact,3,LSTM13_Batch2,0.701792,0.000217
